In [1]:
# Not needed.  Code is embedded in stress modules.
# Adds lag and exponent columns to hist_all.csv and CCARscen_all.csv tables.
# 20170410: expanded to handle CCAR sUE adjustments through 2017
# 20180314: expanded to handle CCAR sUE adjustments through 2018
# creates UE_adj.csv file for sUE calcs
import numpy as np
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import matplotlib.pyplot as plt
import statsmodels.api as sm
from patsy import dmatrices

import sys
import os
os.chdir('C:/BankRisk')

In [2]:
## new file for econ variables only:  add lags, exponents, then save for future use.
cc = pd.read_csv("CCARscen_all.csv")  ## CCAR scenarios for all years
hs = pd.read_csv("hist_all.csv")  ## history of all macroeconomic variables

cc['Date'] = pd.to_datetime(cc['Date'])
hs['Date'] = pd.to_datetime(hs['Date'])

In [3]:
## history vars: 4q lags & 4q differences for UE, HPI, CRE (also, try 6q & 8q lags for UE only)
hs['UE_4'] = hs['UE'].shift(4)  ## add 4q lag version of UE
hs['HPI_4'] = hs['HPI'].shift(4)
hs['CRE_4'] = hs['CRE'].shift(4)

## history vars: squared terms for UE and UE_4:
hs['UE2'] = hs['UE'] * hs['UE']
hs['UE_42'] = hs['UE_4'] * hs['UE_4']

hsx = hs.copy()
hsx[hsx['Date']>'2001-01-01'].tail()

Date  rGDPg  nGDPg  rDIg  nDIg    UE  CPI  T3m  T5y  T10y  ...  \
175 2019-12-31    2.4    3.9   1.9   3.4   3.6  2.4  1.6  1.6   1.8  ...   
176 2020-03-31   -5.0   -3.4   2.6   3.9   3.8  1.2  1.1  1.2   1.4  ...   
177 2020-06-30  -31.4  -32.8  48.5  46.2  13.1 -3.5  0.1  0.4   0.7  ...   
178 2020-09-30   33.4   38.3 -16.3 -13.2   8.8  5.2  0.1  0.3   0.6  ...   
179 2020-12-31    3.7    5.5  -8.1  -7.5   6.8  2.2  0.1  0.4   0.9  ...   

     Prime       DJ    HPI    CRE   VIX  UE_4  HPI_4  CRE_4     UE2  UE_42  
175    4.8  33035.0  213.0  301.0  20.6   3.8  205.0  279.0   12.96  14.44  
176    4.4  25985.0  215.0  302.0  82.7   3.9  207.0  287.0   14.44  15.21  
177    3.3  31577.0  218.0  304.0  57.1   3.7  209.0  301.0  171.61  13.69  
178    3.3  34306.0  223.0  300.0  33.6   3.6  210.0  309.0   77.44  12.96  
179    3.3  39220.0  225.0  297.0  40.3   3.6  213.0  301.0   46.24  12.96  

[5 rows x 22 columns]

In [4]:
hsx.to_csv("hist_lagexp.csv")
hsy = hsx[hsx['Date']>'2001-01-01'].copy()
hsy.set_index('Date',drop=True,inplace=True)
hsy.tail()

rGDPg  nGDPg  rDIg  nDIg    UE  CPI  T3m  T5y  T10y  BBB  ...  \
Date                                                                  ...   
2019-12-31    2.4    3.9   1.9   3.4   3.6  2.4  1.6  1.6   1.8  3.3  ...   
2020-03-31   -5.0   -3.4   2.6   3.9   3.8  1.2  1.1  1.2   1.4  3.4  ...   
2020-06-30  -31.4  -32.8  48.5  46.2  13.1 -3.5  0.1  0.4   0.7  3.4  ...   
2020-09-30   33.4   38.3 -16.3 -13.2   8.8  5.2  0.1  0.3   0.6  2.4  ...   
2020-12-31    3.7    5.5  -8.1  -7.5   6.8  2.2  0.1  0.4   0.9  2.3  ...   

            Prime       DJ    HPI    CRE   VIX  UE_4  HPI_4  CRE_4     UE2  \
Date                                                                         
2019-12-31    4.8  33035.0  213.0  301.0  20.6   3.8  205.0  279.0   12.96   
2020-03-31    4.4  25985.0  215.0  302.0  82.7   3.9  207.0  287.0   14.44   
2020-06-30    3.3  31577.0  218.0  304.0  57.1   3.7  209.0  301.0  171.61   
2020-09-30    3.3  34306.0  223.0  300.0  33.6   3.6  210.0  309.0   77.44   
2020-12-31    3.3  39220.0  225.0  297.0  40.3   3.6  213.0  301.0   46.24   

            UE_42  
Date               
2019-12-31  14.44  
2020-03-31  15.21  
2020-06-30  13.69  
2020-09-30  12.96  
2020-12-31  12.96  

[5 rows x 21 columns]

In [5]:
hsy.columns

Index(['rGDPg', 'nGDPg', 'rDIg', 'nDIg', 'UE', 'CPI', 'T3m', 'T5y', 'T10y',
       'BBB', 'Mort', 'Prime', 'DJ', 'HPI', 'CRE', 'VIX', 'UE_4', 'HPI_4',
       'CRE_4', 'UE2', 'UE_42'],
      dtype='object')

In [6]:
ccy = cc.copy()
ccy['UE_4'] = ccy.groupby(['CCAR','Scen'])[['UE']].shift(4)
ccy['HPI_4'] = ccy.groupby(['CCAR','Scen'])[['HPI']].shift(4)
ccy['CRE_4'] = ccy.groupby(['CCAR','Scen'])[['CRE']].shift(4)
#test.columns = ['UE_4','HPI_4','CRE_4']
ccy.tail(26)

CCAR Scen       Date  rGDPg  nGDPg  rDIg  nDIg    UE  CPI  T3m  ...  BBB  \
380  2021  Adv 2021-06-30   -4.0   -3.4  -9.9  -9.3   8.6  1.0  0.1  ...  5.7   
381  2021  Adv 2021-09-30   -3.3   -2.9  -4.1  -3.4   9.3  1.0  0.1  ...  6.0   
382  2021  Adv 2021-12-31   -1.0   -0.4  -0.6   0.2   9.7  1.2  0.1  ...  6.0   
383  2021  Adv 2022-03-31   -1.0   -0.1   0.3   1.1  10.1  1.2  0.1  ...  6.0   
384  2021  Adv 2022-06-30   -1.0    0.2   0.1   1.1  10.5  1.3  0.1  ...  5.7   
385  2021  Adv 2022-09-30   -0.2    1.0   0.6   1.7  10.8  1.5  0.1  ...  5.4   
386  2021  Adv 2022-12-31    6.7    8.3   5.2   6.5  10.2  1.6  0.1  ...  5.1   
387  2021  Adv 2023-03-31    6.7    8.4   5.5   7.0   9.6  1.7  0.1  ...  4.8   
388  2021  Adv 2023-06-30    6.7    8.5   4.9   6.4   9.1  1.8  0.1  ...  4.5   
389  2021  Adv 2023-09-30    6.7    8.8   4.4   6.1   8.5  2.0  0.1  ...  4.2   
390  2021  Adv 2023-12-31    6.7    8.9   3.9   5.7   8.0  2.1  0.1  ...  3.9   
391  2021  Adv 2024-03-31    6.7    9.0   3.1   5.0   7.4  2.2  0.1  ...  3.6   
392  2021  Sev 2020-12-31    3.7    5.5  -8.1  -7.5   6.8  2.2  0.1  ...  2.3   
393  2021  Sev 2021-03-31   -5.5   -4.8   2.8   3.8   7.8  1.3  0.1  ...  4.9   
394  2021  Sev 2021-06-30   -4.0   -3.4  -9.9  -9.3   8.6  1.0  0.1  ...  5.7   
395  2021  Sev 2021-09-30   -3.3   -2.9  -4.1  -3.4   9.3  1.0  0.1  ...  6.0   
396  2021  Sev 2021-12-31   -1.0   -0.4  -0.6   0.2   9.7  1.2  0.1  ...  6.0   
397  2021  Sev 2022-03-31   -1.0   -0.1   0.3   1.1  10.1  1.2  0.1  ...  6.0   
398  2021  Sev 2022-06-30   -1.0    0.2   0.1   1.1  10.5  1.3  0.1  ...  5.7   
399  2021  Sev 2022-09-30   -0.2    1.0   0.6   1.7  10.8  1.5  0.1  ...  5.4   
400  2021  Sev 2022-12-31    6.7    8.3   5.2   6.5  10.2  1.6  0.1  ...  5.1   
401  2021  Sev 2023-03-31    6.7    8.4   5.5   7.0   9.6  1.7  0.1  ...  4.8   
402  2021  Sev 2023-06-30    6.7    8.5   4.9   6.4   9.1  1.8  0.1  ...  4.5   
403  2021  Sev 2023-09-30    6.7    8.8   4.4   6.1   8.5  2.0  0.1  ...  4.2   
404  2021  Sev 2023-12-31    6.7    8.9   3.9   5.7   8.0  2.1  0.1  ...  3.9   
405  2021  Sev 2024-03-31    6.7    9.0   3.1   5.0   7.4  2.2  0.1  ...  3.6   

     Mort  Prime       DJ    HPI    CRE   VIX  UE_4  HPI_4  CRE_4  
380   3.7    3.2  19178.0  213.0  279.0  65.8   NaN    NaN    NaN  
381   3.8    3.2  17650.0  206.0  267.0  62.0   NaN    NaN    NaN  
382   3.8    3.2  17711.0  198.0  250.0  57.6   6.8  225.0  297.0  
383   3.8    3.2  18626.0  190.0  232.0  55.7   7.8  223.0  288.0  
384   3.7    3.2  20672.0  182.0  214.0  48.6   8.6  213.0  279.0  
385   3.7    3.2  22091.0  175.0  196.0  43.6   9.3  206.0  267.0  
386   3.6    3.2  24004.0  172.0  184.0  39.4   9.7  198.0  250.0  
387   3.6    3.2  25977.0  172.0  178.0  36.3  10.1  190.0  232.0  
388   3.5    3.2  28166.0  175.0  178.0  33.3  10.5  182.0  214.0  
389   3.4    3.2  30649.0  177.0  178.0  30.3  10.8  175.0  196.0  
390   3.4    3.2  33456.0  180.0  178.0  28.3  10.2  172.0  184.0  
391   3.3    3.2  36631.0  183.0  181.0  27.3   9.6  172.0  178.0  
392   2.8    3.3  39220.0  225.0  297.0  40.3   NaN    NaN    NaN  
393   3.5    3.2  23195.0  223.0  288.0  70.0   NaN    NaN    NaN  
394   3.7    3.2  19178.0  213.0  279.0  65.8   NaN    NaN    NaN  
395   3.8    3.2  17650.0  206.0  267.0  62.0   NaN    NaN    NaN  
396   3.8    3.2  17711.0  198.0  250.0  57.6   6.8  225.0  297.0  
397   3.8    3.2  18626.0  190.0  232.0  55.7   7.8  223.0  288.0  
398   3.7    3.2  20672.0  182.0  214.0  48.6   8.6  213.0  279.0  
399   3.7    3.2  22091.0  175.0  196.0  43.6   9.3  206.0  267.0  
400   3.6    3.2  24004.0  172.0  184.0  39.4   9.7  198.0  250.0  
401   3.6    3.2  25977.0  172.0  178.0  36.3  10.1  190.0  232.0  
402   3.5    3.2  28166.0  175.0  178.0  33.3  10.5  182.0  214.0  
403   3.4    3.2  30649.0  177.0  178.0  30.3  10.8  175.0  196.0  
404   3.4    3.2  33456.0  180.0  178.0  28.3  10.2  172.0  184.0  
405   3.3    3.2  36631.0  183.0  181.0  27.3   9

In [7]:
for i in np.arange(len(ccy)):
  if np.isnan(ccy.iloc[i]['UE_4']):
    yr = ccy.iloc[i]['Date'].year - 1
    mn = ccy.iloc[i]['Date'].month
    dy = ccy.iloc[i]['Date'].day
    dd = str(datetime(yr,mn,dy))[0:7]
    xu = hsy[dd]['UE'].values
    ccy.loc[i,'UE_4'] = xu
    xh = hsy[dd]['HPI'].values
    ccy.loc[i,'HPI_4'] = xh
    xc = hsy[dd]['CRE'].values
    ccy.loc[i,'CRE_4'] = xc

ccy.head(8)

CCAR Scen       Date  rGDPg  nGDPg  rDIg  nDIg    UE   CPI  T3m  ...   BBB  \
0  2012  Bas 2011-09-30   0.80   3.30  2.10  4.30  9.00  2.60  0.0  ...  4.90   
1  2012  Bas 2011-12-31   2.33   5.22 -0.48  2.01  9.10  1.90  0.1  ...  4.52   
2  2012  Bas 2012-03-31   1.92   4.76  1.62  3.50  9.10  2.00  0.1  ...  4.55   
3  2012  Bas 2012-06-30   2.22   4.60  2.09  3.88  9.00  1.90  0.1  ...  4.59   
4  2012  Bas 2012-09-30   2.43   4.82  1.99  4.09  8.90  2.20  0.1  ...  4.74   
5  2012  Bas 2012-12-31   2.63   4.77  2.39  4.40  8.90  2.10  0.1  ...  4.89   
6  2012  Bas 2013-03-31   2.69   4.80  2.77  4.80  8.69  2.11  0.1  ...  5.15   
7  2012  Bas 2013-06-30   2.81   5.09  2.89  5.09  8.48  2.27  0.1  ...  5.32   

   Mort  Prime        DJ     HPI     CRE    VIX  UE_4   HPI_4   CRE_4  
0  4.20    NaN  11677.00  132.00  177.00  48.00   9.5  136.00  168.00  
1  4.21    NaN  11936.09  137.21  172.17  35.97   9.5  135.00  168.00  
2  4.17    NaN  12090.22  137.55  173.29  35.08   9.0  134.00  172.00  
3  4.15    NaN  12242.03  137.89  175.43  31.83   9.1  133.00  173.00  
4  4.19    NaN  12401.69  138.24  177.65  31.35   9.0  132.00  177.00  
5  4.24    NaN  12562.14  138.58  178.87  29.59   9.1  137.21  172.17  
6  4.34    NaN  12725.49  138.93  182.28  28.43   9.1  137.55  173.29  
7  4.41    NaN  12899.23  139.28  185.64  30.74   9.0  137.89  175.43  

[8 rows x 22 columns]

In [8]:
ccy['UE2'] = ccy['UE'] * ccy['UE']
ccy['UE_42'] = ccy['UE_4'] * ccy['UE_4']
ccy.to_csv('CCARscen_lagexp.csv')
ccy.head(8)

CCAR Scen       Date  rGDPg  nGDPg  rDIg  nDIg    UE   CPI  T3m  ...  \
0  2012  Bas 2011-09-30   0.80   3.30  2.10  4.30  9.00  2.60  0.0  ...   
1  2012  Bas 2011-12-31   2.33   5.22 -0.48  2.01  9.10  1.90  0.1  ...   
2  2012  Bas 2012-03-31   1.92   4.76  1.62  3.50  9.10  2.00  0.1  ...   
3  2012  Bas 2012-06-30   2.22   4.60  2.09  3.88  9.00  1.90  0.1  ...   
4  2012  Bas 2012-09-30   2.43   4.82  1.99  4.09  8.90  2.20  0.1  ...   
5  2012  Bas 2012-12-31   2.63   4.77  2.39  4.40  8.90  2.10  0.1  ...   
6  2012  Bas 2013-03-31   2.69   4.80  2.77  4.80  8.69  2.11  0.1  ...   
7  2012  Bas 2013-06-30   2.81   5.09  2.89  5.09  8.48  2.27  0.1  ...   

   Prime        DJ     HPI     CRE    VIX  UE_4   HPI_4   CRE_4      UE2  \
0    NaN  11677.00  132.00  177.00  48.00   9.5  136.00  168.00  81.0000   
1    NaN  11936.09  137.21  172.17  35.97   9.5  135.00  168.00  82.8100   
2    NaN  12090.22  137.55  173.29  35.08   9.0  134.00  172.00  82.8100   
3    NaN  12242.03  137.89  175.43  31.83   9.1  133.00  173.00  81.0000   
4    NaN  12401.69  138.24  177.65  31.35   9.0  132.00  177.00  79.2100   
5    NaN  12562.14  138.58  178.87  29.59   9.1  137.21  172.17  79.2100   
6    NaN  12725.49  138.93  182.28  28.43   9.1  137.55  173.29  75.5161   
7    NaN  12899.23  139.28  185.64  30.74   9.0  137.89  175.43  71.9104   

   UE_42  
0  90.25  
1  90.25  
2  81.00  
3  82.81  
4  81.00  
5  82.81  
6  82.81  
7  81.00  

[8 rows x 24 columns]

In [9]:
vv = 'UE'
hsx1 = hsx[hsx['Date']>'2001-01-01']
hsx1.set_index('Date', drop=True, inplace=True)
cc2 = ccy[['Date','CCAR','Scen',vv]]
cc2['YrSc'] = cc2.apply(lambda x: str(x['CCAR'])+x['Scen'],axis=1)
cc2p = cc2.pivot('Date','YrSc',vv)
#cc2p.head()
hsx2 = hsx1.merge(cc2p, how='outer', left_index=True, right_index=True)
#hsx2.tail()

x = list(hsx2.index)

plt.plot(x,hsx2[vv],c='b',label='Actual')
plt.plot(x,hsx2['2012Sev'],ls='-',c='m',lw=.5,marker='.',ms=4,label='2012Sev')
plt.plot(x,hsx2['2013Sev'],ls='--',c='g',lw=.5,marker='+',ms=4,label='2013Sev')
plt.plot(x,hsx2['2014Sev'],ls=':',c='k',lw=1,marker='.',ms=4,label='2014Sev')
plt.plot(x,hsx2['2015Sev'],c='r',lw=.5,marker='d',ms=4,label='2015Sev')
plt.plot(x,hsx2['2015Adv'],c='r',lw=.5,marker='x',ms=4,label='2015Adv')
plt.plot(x,hsx2['2015Bas'],c='r',lw=.5,label='2015Bas')
plt.title(vv)
plt.legend(loc='center left', bbox_to_anchor=(1,0.5))
plt.grid(b=True, which='major', color='k', linestyle=':')
plt.savefig(vv+'_chart.png', dpi=400, bbox_inches='tight')
plt.show()

In [10]:
hsue = pd.DataFrame(hsy.loc['2001':'2020']['UE'])
hsue.reset_index(drop=False, inplace=True)
hsue.tail()

Date    UE
75 2019-12-31   3.6
76 2020-03-31   3.8
77 2020-06-30  13.1
78 2020-09-30   8.8
79 2020-12-31   6.8

In [11]:
ccs = ccy[ccy['Scen']=='Sev'][['CCAR','Date','UE']]
ccs = ccs.pivot('Date','CCAR','UE')
ccs.reset_index(drop=False, inplace=True)
ccs.columns = ['Date','u12','u13','u14','u15','u16','u17','u18','u19','u20','u21']
ccs.tail(8)

Date  u12  u13  u14  u15  u16  u17  u18  u19  u20   u21
43 2022-06-30  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  9.5  10.5
44 2022-09-30  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  9.2  10.8
45 2022-12-31  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  8.8  10.2
46 2023-03-31  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  8.5   9.6
47 2023-06-30  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   9.1
48 2023-09-30  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   8.5
49 2023-12-31  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   8.0
50 2024-03-31  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   7.4

In [12]:
hsue2 = hsue.merge(ccs,how='outer',on='Date')
hsue2.set_index('Date', drop=True, inplace=True)
hsue2.tail(18)

UE  u12  u13  u14  u15  u16  u17  u18   u19   u20   u21
Date                                                                 
2019-12-31   3.6  NaN  NaN  NaN  NaN  NaN  9.1  9.9   8.4   3.5   NaN
2020-03-31   3.8  NaN  NaN  NaN  NaN  NaN  8.9  9.7   9.2   4.5   NaN
2020-06-30  13.1  NaN  NaN  NaN  NaN  NaN  NaN  9.5   9.7   6.1   NaN
2020-09-30   8.8  NaN  NaN  NaN  NaN  NaN  NaN  9.2  10.0   7.4   NaN
2020-12-31   6.8  NaN  NaN  NaN  NaN  NaN  NaN  8.9   9.9   8.4   6.8
2021-03-31   NaN  NaN  NaN  NaN  NaN  NaN  NaN  8.6   9.7   9.2   7.8
2021-06-30   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   9.5   9.7   8.6
2021-09-30   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   9.2  10.0   9.3
2021-12-31   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   8.9   9.9   9.7
2022-03-31   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   8.6   9.7  10.1
2022-06-30   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   9.5  10.5
2022-09-30   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   9.2  10.8
2022-12-31   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   8.8  10.2
2023-03-31   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   8.5   9.6
2023-06-30   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   9.1
2023-09-30   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   8.5
2023-12-31   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   8.0
2024-03-31   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   7.4

In [13]:
hsue2['Qn'] = hsue2.index.year * 4 + hsue2.index.month /3
hsue2['Qn'] = hsue2['Qn'].astype(int)
hsue2.tail(16)

UE  u12  u13  u14  u15  u16  u17  u18   u19   u20   u21    Qn
Date                                                                       
2020-06-30  13.1  NaN  NaN  NaN  NaN  NaN  NaN  9.5   9.7   6.1   NaN  8082
2020-09-30   8.8  NaN  NaN  NaN  NaN  NaN  NaN  9.2  10.0   7.4   NaN  8083
2020-12-31   6.8  NaN  NaN  NaN  NaN  NaN  NaN  8.9   9.9   8.4   6.8  8084
2021-03-31   NaN  NaN  NaN  NaN  NaN  NaN  NaN  8.6   9.7   9.2   7.8  8085
2021-06-30   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   9.5   9.7   8.6  8086
2021-09-30   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   9.2  10.0   9.3  8087
2021-12-31   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   8.9   9.9   9.7  8088
2022-03-31   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   8.6   9.7  10.1  8089
2022-06-30   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   9.5  10.5  8090
2022-09-30   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   9.2  10.8  8091
2022-12-31   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   8.8  10.2  8092
2023-03-31   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   8.5   9.6  8093
2023-06-30   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   9.1  8094
2023-09-30   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   8.5  8095
2023-12-31   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   8.0  8096
2024-03-31   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   7.4  8097

In [14]:
mx = hsue2[['UE','u12','u13','u14','u15','u16','u17','u18','u19','u20','u21']].max()
mx = pd.DataFrame(mx)
mx.columns = ['mx']
mxdct = mx.to_dict()
mxdct['mx']

{'UE': 13.1,
 'u12': 13.05,
 'u13': 12.1,
 'u14': 11.3,
 'u15': 10.1,
 'u16': 10.0,
 'u17': 10.0,
 'u18': 10.0,
 'u19': 10.0,
 'u20': 10.0,
 'u21': 10.8}

In [15]:
## remove pandemic UE value of 13.1 (and replace with financial crisis max value of 9.9)
##  this is required to prevent huge distortions caused by pandemic shut-down
mxdct['mx']['UE']=9.9
mxdct['mx']

{'UE': 9.9,
 'u12': 13.05,
 'u13': 12.1,
 'u14': 11.3,
 'u15': 10.1,
 'u16': 10.0,
 'u17': 10.0,
 'u18': 10.0,
 'u19': 10.0,
 'u20': 10.0,
 'u21': 10.8}

In [16]:
mxd = hsue2[hsue2['UE']==mxdct['mx']['UE']]['Qn']
mxd12 = hsue2[hsue2['u12']==mxdct['mx']['u12']]['Qn']
mxd13 = hsue2[hsue2['u13']==mxdct['mx']['u13']]['Qn']
mxd14 = hsue2[hsue2['u14']==mxdct['mx']['u14']]['Qn']
mxd15 = hsue2[hsue2['u15']==mxdct['mx']['u15']]['Qn']
mxd16 = hsue2[hsue2['u16']==mxdct['mx']['u16']]['Qn']
mxd17 = hsue2[hsue2['u17']==mxdct['mx']['u17']]['Qn']
mxd18 = hsue2[hsue2['u18']==mxdct['mx']['u18']]['Qn']
mxd19 = hsue2[hsue2['u19']==mxdct['mx']['u19']]['Qn']
mxd20 = hsue2[hsue2['u20']==mxdct['mx']['u20']]['Qn']
mxd21 = hsue2[hsue2['u21']==mxdct['mx']['u21']]['Qn']
mxd15, mxd21

(Date
 2016-06-30    8066
 Name: Qn, dtype: int32, Date
 2022-09-30    8091
 Name: Qn, dtype: int32)

In [17]:
mxq = pd.DataFrame(pd.concat([mxd,mxd12,mxd13,mxd14,mxd15,mxd16,mxd17,mxd18,mxd19,mxd20,mxd21]))
mxq.reset_index(drop=False, inplace=True)
ux=pd.Series(['UE','u12','u13','u14','u15','u16','u17','u18','u19','u20','u21'])
mxq = mxq.assign(ux=ux.values)
mxq.set_index('ux',drop=True,inplace=True)
mxq

Date    Qn
ux                  
UE  2009-12-31  8040
u12 2013-06-30  8054
u13 2014-06-30  8058
u14 2015-06-30  8062
u15 2016-06-30  8066
u16 2017-09-30  8071
u17 2018-09-30  8075
u18 2019-09-30  8079
u19 2020-09-30  8083
u20 2021-09-30  8087
u21 2022-09-30  8091

In [18]:
mx2 = mx.merge(mxq, how='inner', left_index=True, right_index=True)
#mx2.reset_index(drop=False,inplace=True)
mx2.at['UE','mx']=9.9 ### new syntax using '.at' so that original df is altered

In [19]:
mx2

mx       Date    Qn
UE    9.90 2009-12-31  8040
u12  13.05 2013-06-30  8054
u13  12.10 2014-06-30  8058
u14  11.30 2015-06-30  8062
u15  10.10 2016-06-30  8066
u16  10.00 2017-09-30  8071
u17  10.00 2018-09-30  8075
u18  10.00 2019-09-30  8079
u19  10.00 2020-09-30  8083
u20  10.00 2021-09-30  8087
u21  10.80 2022-09-30  8091

In [20]:
hmx = mx2.at['UE','mx']
hqn = mx2.at['UE','Qn']

# calcs incremental adjustment to curr UE rate per period:
for i in np.arange(len(mx2)):
  mx2.at[mx2.index[i],'adj'] = (1 - hmx / mx2.at[mx2.index[i],'mx']  )
  mx2.at[mx2.index[i],'inc'] = (1 - hmx / mx2.at[mx2.index[i],'mx']  )  /(mx2.at[mx2.index[i],'Qn'] - hqn)

mx2

mx       Date    Qn       adj       inc
UE    9.90 2009-12-31  8040  0.000000       NaN
u12  13.05 2013-06-30  8054  0.241379  0.017241
u13  12.10 2014-06-30  8058  0.181818  0.010101
u14  11.30 2015-06-30  8062  0.123894  0.005632
u15  10.10 2016-06-30  8066  0.019802  0.000762
u16  10.00 2017-09-30  8071  0.010000  0.000323
u17  10.00 2018-09-30  8075  0.010000  0.000286
u18  10.00 2019-09-30  8079  0.010000  0.000256
u19  10.00 2020-09-30  8083  0.010000  0.000233
u20  10.00 2021-09-30  8087  0.010000  0.000213
u21  10.80 2022-09-30  8091  0.083333  0.001634

In [21]:
hsue2.to_csv('uescaledata.csv')

In [22]:
mx2.to_csv("uescaleparam.csv")

In [23]:
ues = pd.DataFrame(hsue2['Qn'])
ues['adj'] = 1
ues.reset_index(drop=False, inplace=True)
ues.head()

Date    Qn  adj
0 2001-03-31  8005    1
1 2001-06-30  8006    1
2 2001-09-30  8007    1
3 2001-12-31  8008    1
4 2002-03-31  8009    1

In [24]:
## 2012 adjustment:
ues['adj'] = 1.0
ast = pd.to_datetime(mx2.at['UE','Date'])
aen = pd.to_datetime(mx2.at['u12','Date'])

inc = np.float(mx2.at['u12','inc'])
adj = np.float(1.0 - mx2.at['u12','adj'])
cadj = 1.0
for i in np.arange(len(ues)):
  if pd.to_datetime(ues.at[i,'Date']) >= aen:
    ues.at[i,'adj'] = adj
  elif pd.to_datetime(ues.at[i,'Date']) > ast:
    cadj = cadj - inc
    ues.at[i,'adj'] = cadj

ues.rename(columns={'adj':'adj12'}, inplace=True)
ues.tail()

Date    Qn     adj12
88 2023-03-31  8093  0.758621
89 2023-06-30  8094  0.758621
90 2023-09-30  8095  0.758621
91 2023-12-31  8096  0.758621
92 2024-03-31  8097  0.758621

In [25]:
## 2013 adjustment:
ues['adj'] = 1.0
ast = pd.to_datetime(mx2.at['UE','Date'])
aen = pd.to_datetime(mx2.at['u13','Date'])

inc = np.float(mx2.at['u13','inc'])
adj = np.float(1.0 - mx2.at['u13','adj'])
cadj = 1.0
for i in np.arange(len(ues)):
  if pd.to_datetime(ues.at[i,'Date']) >= aen:
    ues.at[i,'adj'] = adj
  elif pd.to_datetime(ues.at[i,'Date']) > ast:
    cadj = cadj - inc
    ues.at[i,'adj'] = cadj

ues.rename(columns={'adj':'adj13'}, inplace=True)
ues.tail()

Date    Qn     adj12     adj13
88 2023-03-31  8093  0.758621  0.818182
89 2023-06-30  8094  0.758621  0.818182
90 2023-09-30  8095  0.758621  0.818182
91 2023-12-31  8096  0.758621  0.818182
92 2024-03-31  8097  0.758621  0.818182

In [26]:
## 2014 adjustment:
ues['adj'] = 1.0
ast = pd.to_datetime(mx2.at['UE','Date'])
aen = pd.to_datetime(mx2.at['u14','Date'])

inc = np.float(mx2.at['u14','inc'])
adj = np.float(1.0 - mx2.at['u14','adj'])
cadj = 1.0
for i in np.arange(len(ues)):
  if pd.to_datetime(ues.at[i,'Date']) >= aen:
    ues.at[i,'adj'] = adj
  elif pd.to_datetime(ues.at[i,'Date']) > ast:
    cadj = cadj - inc
    ues.at[i,'adj'] = cadj

ues.rename(columns={'adj':'adj14'}, inplace=True)
ues.tail()

Date    Qn     adj12     adj13     adj14
88 2023-03-31  8093  0.758621  0.818182  0.876106
89 2023-06-30  8094  0.758621  0.818182  0.876106
90 2023-09-30  8095  0.758621  0.818182  0.876106
91 2023-12-31  8096  0.758621  0.818182  0.876106
92 2024-03-31  8097  0.758621  0.818182  0.876106

In [27]:
## 2015 adjustment:
ues['adj'] = 1.0
ast = pd.to_datetime(mx2.at['UE','Date'])
aen = pd.to_datetime(mx2.at['u15','Date'])

inc = np.float(mx2.at['u15','inc'])
adj = np.float(1.0 - mx2.at['u15','adj'])
cadj = 1.0
for i in np.arange(len(ues)):
  if pd.to_datetime(ues.at[i,'Date']) >= aen:
    ues.at[i,'adj'] = adj
  elif pd.to_datetime(ues.at[i,'Date']) > ast:
    cadj = cadj - inc
    ues.at[i,'adj'] = cadj

ues.rename(columns={'adj':'adj15'}, inplace=True)
ues.tail()

Date    Qn     adj12     adj13     adj14     adj15
88 2023-03-31  8093  0.758621  0.818182  0.876106  0.980198
89 2023-06-30  8094  0.758621  0.818182  0.876106  0.980198
90 2023-09-30  8095  0.758621  0.818182  0.876106  0.980198
91 2023-12-31  8096  0.758621  0.818182  0.876106  0.980198
92 2024-03-31  8097  0.758621  0.818182  0.876106  0.980198

In [28]:
## 2016 adjustment:
ues['adj'] = 1.0
ast = pd.to_datetime(mx2.at['UE','Date'])
aen = pd.to_datetime(mx2.at['u16','Date'])

inc = np.float(mx2.at['u16','inc'])
adj = np.float(1.0 - mx2.at['u16','adj'])
cadj = 1.0
for i in np.arange(len(ues)):
  if pd.to_datetime(ues.at[i,'Date']) >= aen:
    ues.at[i,'adj'] = adj
  elif pd.to_datetime(ues.at[i,'Date']) > ast:
    cadj = cadj - inc
    ues.at[i,'adj'] = cadj

ues.rename(columns={'adj':'adj16'}, inplace=True)
ues.tail()

Date    Qn     adj12     adj13     adj14     adj15  adj16
88 2023-03-31  8093  0.758621  0.818182  0.876106  0.980198   0.99
89 2023-06-30  8094  0.758621  0.818182  0.876106  0.980198   0.99
90 2023-09-30  8095  0.758621  0.818182  0.876106  0.980198   0.99
91 2023-12-31  8096  0.758621  0.818182  0.876106  0.980198   0.99
92 2024-03-31  8097  0.758621  0.818182  0.876106  0.980198   0.99

In [29]:
## 2017 adjustment:
ues['adj'] = 1.0
ast = pd.to_datetime(mx2.at['UE','Date'])
aen = pd.to_datetime(mx2.at['u17','Date'])

inc = np.float(mx2.at['u17','inc'])
adj = np.float(1.0 - mx2.at['u17','adj'])
cadj = 1.0
for i in np.arange(len(ues)):
  if pd.to_datetime(ues.at[i,'Date']) >= aen:
    ues.at[i,'adj'] = adj
  elif pd.to_datetime(ues.at[i,'Date']) > ast:
    cadj = cadj - inc
    ues.at[i,'adj'] = cadj

ues.rename(columns={'adj':'adj17'}, inplace=True)
ues.tail()

Date    Qn     adj12     adj13     adj14     adj15  adj16  adj17
88 2023-03-31  8093  0.758621  0.818182  0.876106  0.980198   0.99   0.99
89 2023-06-30  8094  0.758621  0.818182  0.876106  0.980198   0.99   0.99
90 2023-09-30  8095  0.758621  0.818182  0.876106  0.980198   0.99   0.99
91 2023-12-31  8096  0.758621  0.818182  0.876106  0.980198   0.99   0.99
92 2024-03-31  8097  0.758621  0.818182  0.876106  0.980198   0.99   0.99

In [30]:
## 2018 adjustment:
ues['adj'] = 1.0
ast = pd.to_datetime(mx2.at['UE','Date'])
aen = pd.to_datetime(mx2.at['u18','Date'])

inc = np.float(mx2.at['u18','inc'])
adj = np.float(1.0 - mx2.at['u18','adj'])
cadj = 1.0
for i in np.arange(len(ues)):
  if pd.to_datetime(ues.at[i,'Date']) >= aen:
    ues.at[i,'adj'] = adj
  elif pd.to_datetime(ues.at[i,'Date']) > ast:
    cadj = cadj - inc
    ues.at[i,'adj'] = cadj

ues.rename(columns={'adj':'adj18'}, inplace=True)
ues.tail()

Date    Qn     adj12     adj13     adj14     adj15  adj16  adj17  \
88 2023-03-31  8093  0.758621  0.818182  0.876106  0.980198   0.99   0.99   
89 2023-06-30  8094  0.758621  0.818182  0.876106  0.980198   0.99   0.99   
90 2023-09-30  8095  0.758621  0.818182  0.876106  0.980198   0.99   0.99   
91 2023-12-31  8096  0.758621  0.818182  0.876106  0.980198   0.99   0.99   
92 2024-03-31  8097  0.758621  0.818182  0.876106  0.980198   0.99   0.99   

    adj18  
88   0.99  
89   0.99  
90   0.99  
91   0.99  
92   0.99

In [31]:
## 2019 adjustment:
ues['adj'] = 1.0
ast = pd.to_datetime(mx2.at['UE','Date'])
aen = pd.to_datetime(mx2.at['u19','Date'])

inc = np.float(mx2.at['u19','inc'])
adj = np.float(1.0 - mx2.at['u19','adj'])
cadj = 1.0
for i in np.arange(len(ues)):
  if pd.to_datetime(ues.at[i,'Date']) >= aen:
    ues.at[i,'adj'] = adj
  elif pd.to_datetime(ues.at[i,'Date']) > ast:
    cadj = cadj - inc
    ues.at[i,'adj'] = cadj

ues.rename(columns={'adj':'adj19'}, inplace=True)
ues.tail()

Date    Qn     adj12     adj13     adj14     adj15  adj16  adj17  \
88 2023-03-31  8093  0.758621  0.818182  0.876106  0.980198   0.99   0.99   
89 2023-06-30  8094  0.758621  0.818182  0.876106  0.980198   0.99   0.99   
90 2023-09-30  8095  0.758621  0.818182  0.876106  0.980198   0.99   0.99   
91 2023-12-31  8096  0.758621  0.818182  0.876106  0.980198   0.99   0.99   
92 2024-03-31  8097  0.758621  0.818182  0.876106  0.980198   0.99   0.99   

    adj18  adj19  
88   0.99   0.99  
89   0.99   0.99  
90   0.99   0.99  
91   0.99   0.99  
92   0.99   0.99

In [32]:
## 2020 adjustment:
ues['adj'] = 1.0
ast = pd.to_datetime(mx2.at['UE','Date'])
aen = pd.to_datetime(mx2.at['u20','Date'])

inc = np.float(mx2.at['u20','inc'])
adj = np.float(1.0 - mx2.at['u20','adj'])
cadj = 1.0
for i in np.arange(len(ues)):
  if pd.to_datetime(ues.at[i,'Date']) >= aen:
    ues.at[i,'adj'] = adj
  elif pd.to_datetime(ues.at[i,'Date']) > ast:
    cadj = cadj - inc
    ues.at[i,'adj'] = cadj

ues.rename(columns={'adj':'adj20'}, inplace=True)
ues.tail()

Date    Qn     adj12     adj13     adj14     adj15  adj16  adj17  \
88 2023-03-31  8093  0.758621  0.818182  0.876106  0.980198   0.99   0.99   
89 2023-06-30  8094  0.758621  0.818182  0.876106  0.980198   0.99   0.99   
90 2023-09-30  8095  0.758621  0.818182  0.876106  0.980198   0.99   0.99   
91 2023-12-31  8096  0.758621  0.818182  0.876106  0.980198   0.99   0.99   
92 2024-03-31  8097  0.758621  0.818182  0.876106  0.980198   0.99   0.99   

    adj18  adj19  adj20  
88   0.99   0.99   0.99  
89   0.99   0.99   0.99  
90   0.99   0.99   0.99  
91   0.99   0.99   0.99  
92   0.99   0.99   0.99

In [33]:
## 2121 adjustment:
ues['adj'] = 1.0
ast = pd.to_datetime(mx2.at['UE','Date'])
aen = pd.to_datetime(mx2.at['u21','Date'])

inc = np.float(mx2.at['u21','inc'])
adj = np.float(1.0 - mx2.at['u21','adj'])
cadj = 1.0
for i in np.arange(len(ues)):
  if pd.to_datetime(ues.at[i,'Date']) >= aen:
    ues.at[i,'adj'] = adj
  elif pd.to_datetime(ues.at[i,'Date']) > ast:
    cadj = cadj - inc
    ues.at[i,'adj'] = cadj

ues.rename(columns={'adj':'adj21'}, inplace=True)
ues.tail()

Date    Qn     adj12     adj13     adj14     adj15  adj16  adj17  \
88 2023-03-31  8093  0.758621  0.818182  0.876106  0.980198   0.99   0.99   
89 2023-06-30  8094  0.758621  0.818182  0.876106  0.980198   0.99   0.99   
90 2023-09-30  8095  0.758621  0.818182  0.876106  0.980198   0.99   0.99   
91 2023-12-31  8096  0.758621  0.818182  0.876106  0.980198   0.99   0.99   
92 2024-03-31  8097  0.758621  0.818182  0.876106  0.980198   0.99   0.99   

    adj18  adj19  adj20     adj21  
88   0.99   0.99   0.99  0.916667  
89   0.99   0.99   0.99  0.916667  
90   0.99   0.99   0.99  0.916667  
91   0.99   0.99   0.99  0.916667  
92   0.99   0.99   0.99  0.916667

In [34]:
ues.drop('Qn', axis=1, inplace=True)
ues.tail()

Date     adj12     adj13     adj14     adj15  adj16  adj17  adj18  \
88 2023-03-31  0.758621  0.818182  0.876106  0.980198   0.99   0.99   0.99   
89 2023-06-30  0.758621  0.818182  0.876106  0.980198   0.99   0.99   0.99   
90 2023-09-30  0.758621  0.818182  0.876106  0.980198   0.99   0.99   0.99   
91 2023-12-31  0.758621  0.818182  0.876106  0.980198   0.99   0.99   0.99   
92 2024-03-31  0.758621  0.818182  0.876106  0.980198   0.99   0.99   0.99   

    adj19  adj20     adj21  
88   0.99   0.99  0.916667  
89   0.99   0.99  0.916667  
90   0.99   0.99  0.916667  
91   0.99   0.99  0.916667  
92   0.99   0.99  0.916667

In [35]:
ues.to_csv("UE_adj.csv")

In [36]:
ues[35:64]

Date     adj12     adj13     adj14     adj15     adj16     adj17  \
35 2009-12-31  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000   
36 2010-03-31  0.982759  0.989899  0.994368  0.999238  0.999677  0.999714   
37 2010-06-30  0.965517  0.979798  0.988737  0.998477  0.999355  0.999429   
38 2010-09-30  0.948276  0.969697  0.983105  0.997715  0.999032  0.999143   
39 2010-12-31  0.931034  0.959596  0.977474  0.996954  0.998710  0.998857   
40 2011-03-31  0.913793  0.949495  0.971842  0.996192  0.998387  0.998571   
41 2011-06-30  0.896552  0.939394  0.966211  0.995430  0.998065  0.998286   
42 2011-09-30  0.879310  0.929293  0.960579  0.994669  0.997742  0.998000   
43 2011-12-31  0.862069  0.919192  0.954948  0.993907  0.997419  0.997714   
44 2012-03-31  0.844828  0.909091  0.949316  0.993145  0.997097  0.997429   
45 2012-06-30  0.827586  0.898990  0.943685  0.992384  0.996774  0.997143   
46 2012-09-30  0.810345  0.888889  0.938053  0.991622  0.996452  0.996857   
47 2012-12-31  0.793103  0.878788  0.932422  0.990861  0.996129  0.996571   
48 2013-03-31  0.775862  0.868687  0.926790  0.990099  0.995806  0.996286   
49 2013-06-30  0.758621  0.858586  0.921158  0.989337  0.995484  0.996000   
50 2013-09-30  0.758621  0.848485  0.915527  0.988576  0.995161  0.995714   
51 2013-12-31  0.758621  0.838384  0.909895  0.987814  0.994839  0.995429   
52 2014-03-31  0.758621  0.828283  0.904264  0.987053  0.994516  0.995143   
53 2014-06-30  0.758621  0.818182  0.898632  0.986291  0.994194  0.994857   
54 2014-09-30  0.758621  0.818182  0.893001  0.985529  0.993871  0.994571   
55 2014-12-31  0.758621  0.818182  0.887369  0.984768  0.993548  0.994286   
56 2015-03-31  0.758621  0.818182  0.881738  0.984006  0.993226  0.994000   
57 2015-06-30  0.758621  0.818182  0.876106  0.983244  0.992903  0.993714   
58 2015-09-30  0.758621  0.818182  0.876106  0.982483  0.992581  0.993429   
59 2015-12-31  0.758621  0.818182  0.876106  0.981721  0.992258  0.993143   
60 2016-03-31  0.758621  0.818182  0.876106  0.980960  0.991935  0.992857   
61 2016-06-30  0.758621  0.818182  0.876106  0.980198  0.991613  0.992571   
62 2016-09-30  0.758621  0.818182  0.876106  0.980198  0.991290  0.992286   
63 2016-12-31  0.758621  0.818182  0.876106  0.980198  0.990968  0.992000   

       adj18     adj19     adj20     adj21  
35  1.000000  1.000000  1.000000  1.000000  
36  0.999744  0.999767  0.999787  0.998366  
37  0.999487  0.999535  0.999574  0.996732  
38  0.999231  0.999302  0.999362  0.995098  
39  0.998974  0.999070  0.999149  0.993464  
40  0.998718  0.998837  0.998936  0.991830  
41  0.998462  0.998605  0.998723  0.990196  
42  0.998205  0.998372  0.998511  0.988562  
43  0.997949  0.998140  0.998298  0.986928  
44  0.997692  0.997907  0.998085  0.985294  
45  0.997436  0.997674  0.997872  0.983660  
46  0.997179  0.997442  0.997660  0.982026  
47  0.996923  0.997209  0.997447  0.980392  
48  0.996667  0.996977  0.997234  0.978758  
49  0.996410  0.996744  0.997021  0.977124  
50  0.996154  0.996512  0.996809  0.975490  
51  0.995897  0.996279  0.996596  0.973856  
52  0.995641  0.996047  0.996383  0.972222  
53  0.995385  0.995814  0.996170  0.970588  
54  0.995128  0.995581  0.995957  0.968954  
55  0.994872  0.995349  0.995745  0.967320  
56  0.994615  0.995116  0.995532  0.965686  
57  0.994359  0.994884  0.995319  0.964052  
58  0.994103  0.994651  0.995106  0.962418  
59  0.993846  0.994419  0.994894  0.960784  
60  0.993590  0.994186  0.994681  0.959150  
61  0.993333  0.993953  0.994468  0.957516  
62  0.993077  0.993721  0.994255  0.955882  
63  0.992821  0.993488  0.994043  0.954248